In [25]:
import nltk
nltk.download('vader_lexicon')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# analyzer = SentimentIntensityAnalyzer()

%pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mariaelena/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Note: you may need to restart the kernel to use updated packages.


In [28]:
# Load the muse_v3.csv into a DataFrame from the "data" file.

music_data = os.path.join("../data/muse_v3.csv")
music_df = pd.read_csv(music_data)
music_df.head()





,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre
0,https://www.last.fm/music/eminem/_/%2527till%2...,'Till I Collapse,Eminem,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap
1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin',Rick Ross,['aggressive'],1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,['aggressive'],7,3.771176,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal


In [31]:
# load the Resources/lyrics_data_cleaned.csv into a DataFrame and print the head
lyrics_data = os.path.join("../data/lyrics_data_cleaned.csv")
lyrics_df = pd.read_csv(lyrics_data, sep=';')
lyrics_df.head()

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics
0,https://www.last.fm/music/the%2bbeatles/_/till...,Till There Was You,The Beatles,nostalgic,9,6.905870,4.741304,5.758913,0cb9fb3e-441a-4b3b-929a-d3f0746c75c8,0ESIjVxnDnCDaTPo6sStHm,classic rock,\nThere were bells on a hill\nBut I never hear...
1,https://www.last.fm/music/the%2bsundays/_/cry,Cry,The Sundays,intimate,8,6.560987,4.134408,5.448947,21d0be41-bc82-4d6a-9c33-0b51f0cd67aa,16MYd5ZifPer1ijvXYTFVY,indie,\nAnd I'm standing on a platform\nNow I'm star...
2,https://www.last.fm/music/the%2bnational/_/gue...,Guest Room,The National,bittersweet,5,4.109459,2.989730,3.900541,d555fc42-8ae0-4617-965c-062292dbf8b2,2oCtEQQ6BezDWy8qPxo6Wk,indie rock,\nThey're gonna send us to prison for jerks\nF...
3,https://www.last.fm/music/jenny%2blewis%2bwith...,Melt Your Heart,Jenny Lewis with The Watson Twins,wistful,18,5.984598,3.605575,5.521322,cad86013-7dd3-4d5b-9395-b01ba52d02cb,NaN,indie,Nothing is ever as good as it was\nAnd what's ...
4,https://www.last.fm/music/aaron%2backerson/_/l...,LET'S HAVE SUCH A COFFEE,Aaron Ackerson,exciting,7,7.097143,5.487143,6.247143,NaN,1NIOUFJqP7Q5ioetBXmJad,comedy,You become pretty when you draw near love\nLet...


In [32]:
# add a column to the lyrics DataFrame to store the sentiment scores
lyrics_df['compound'] = 0.0
lyrics_df['positive'] = 0.0
lyrics_df['negative'] = 0.0
lyrics_df['neutral'] = 0.0
lyrics_df.head()



,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,compound,positive,negative,neutral
0,https://www.last.fm/music/the%2bbeatles/_/till...,Till There Was You,The Beatles,nostalgic,9,6.905870,4.741304,5.758913,0cb9fb3e-441a-4b3b-929a-d3f0746c75c8,0ESIjVxnDnCDaTPo6sStHm,classic rock,\nThere were bells on a hill\nBut I never hear...,0.0,0.0,0.0,0.0
1,https://www.last.fm/music/the%2bsundays/_/cry,Cry,The Sundays,intimate,8,6.560987,4.134408,5.448947,21d0be41-bc82-4d6a-9c33-0b51f0cd67aa,16MYd5ZifPer1ijvXYTFVY,indie,\nAnd I'm standing on a platform\nNow I'm star...,0.0,0.0,0.0,0.0
2,https://www.last.fm/music/the%2bnational/_/gue...,Guest Room,The National,bittersweet,5,4.109459,2.989730,3.900541,d555fc42-8ae0-4617-965c-062292dbf8b2,2oCtEQQ6BezDWy8qPxo6Wk,indie rock,\nThey're gonna send us to prison for jerks\nF...,0.0,0.0,0.0,0.0
3,https://www.last.fm/music/jenny%2blewis%2bwith...,Melt Your Heart,Jenny Lewis with The Watson Twins,wistful,18,5.984598,3.605575,5.521322,cad86013-7dd3-4d5b-9395-b01ba52d02cb,NaN,indie,Nothing is ever as good as it was\nAnd what's ...,0.0,0.0,0.0,0.0
4,https://www.last.fm/music/aaron%2backerson/_/l...,LET'S HAVE SUCH A COFFEE,Aaron Ackerson,exciting,7,7.097143,5.487143,6.247143,NaN,1NIOUFJqP7Q5ioetBXmJad,comedy,You become pretty when you draw near love\nLet...,0.0,0.0,0.0,0.0


In [33]:
# loop through the lyrics DataFrame and calculate the sentiment scores
for index, row in lyrics_df.iterrows():
    text = row['lyrics']
    sentiment = analyzer.polarity_scores(text)
    lyrics_df.at[index, 'compound'] = sentiment['compound']
    lyrics_df.at[index, 'positive'] = sentiment['pos']
    lyrics_df.at[index, 'negative'] = sentiment['neg']
    lyrics_df.at[index, 'neutral'] = sentiment['neu']

lyrics_df.head()

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,compound,positive,negative,neutral
0,https://www.last.fm/music/the%2bbeatles/_/till...,Till There Was You,The Beatles,nostalgic,9,6.905870,4.741304,5.758913,0cb9fb3e-441a-4b3b-929a-d3f0746c75c8,0ESIjVxnDnCDaTPo6sStHm,classic rock,\nThere were bells on a hill\nBut I never hear...,0.9735,0.196,0.074,0.731
1,https://www.last.fm/music/the%2bsundays/_/cry,Cry,The Sundays,intimate,8,6.560987,4.134408,5.448947,21d0be41-bc82-4d6a-9c33-0b51f0cd67aa,16MYd5ZifPer1ijvXYTFVY,indie,\nAnd I'm standing on a platform\nNow I'm star...,-0.9796,0.000,0.178,0.822
2,https://www.last.fm/music/the%2bnational/_/gue...,Guest Room,The National,bittersweet,5,4.109459,2.989730,3.900541,d555fc42-8ae0-4617-965c-062292dbf8b2,2oCtEQQ6BezDWy8qPxo6Wk,indie rock,\nThey're gonna send us to prison for jerks\nF...,-0.9571,0.017,0.148,0.834
3,https://www.last.fm/music/jenny%2blewis%2bwith...,Melt Your Heart,Jenny Lewis with The Watson Twins,wistful,18,5.984598,3.605575,5.521322,cad86013-7dd3-4d5b-9395-b01ba52d02cb,NaN,indie,Nothing is ever as good as it was\nAnd what's ...,0.5806,0.168,0.140,0.692
4,https://www.last.fm/music/aaron%2backerson/_/l...,LET'S HAVE SUCH A COFFEE,Aaron Ackerson,exciting,7,7.097143,5.487143,6.247143,NaN,1NIOUFJqP7Q5ioetBXmJad,comedy,You become pretty when you draw near love\nLet...,0.9951,0.219,0.029,0.752


In [34]:
# Provide a summarry for each sentiment score
lyrics_df.describe()

# Give a count for songs that fall into each sentiment category
lyrics_df['sentiment'] = 'neutral'
lyrics_df.loc[lyrics_df['compound'] > 0.1, 'sentiment'] = 'positive'
lyrics_df.loc[lyrics_df['compound'] < -0.1, 'sentiment'] = 'negative'
lyrics_df['sentiment'].value_counts()


sentiment
positive    1891
negative    1175
neutral       88
Name: count, dtype: int64

In [7]:
#Creating a function that takes in a song lyric and returns the sentiment score

def analyze_sentiment(lyrics): 
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(lyrics)
    return sentiment_scores
#create a gr interface that allows users to input a song lyric and get the sentiment score
import gradio as gr
iface = gr.Interface(fn=analyze_sentiment, inputs="textbox", outputs="text")
iface.launch(share=True)

interface = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.inputs.Textbox(), 
    outputs="json", 
    title="Song Lyric Sentiment Analysis",
    description="Enter a song lyric to see the sentiment score.",
)

if __name__ == "__main__":
    interface.launch(share=True)



Running on local URL:  http://127.0.0.1:7865
IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://d8fb399515913f17c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


AttributeError: module 'gradio' has no attribute 'inputs'

In [4]:
import gradio as gr
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# import nltk
# nltk.download('vader_lexicon')

# Sample get_lyrics function for demonstration
def get_lyrics(song_name):
    # For the purpose of this example, let's return a dummy lyric
    return "This is a sample lyric of the song."

# Initialize VADER sentiment intensity analyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

def analyze_song_sentiment(song_name):
    # Get lyrics using the provided function
    lyrics = get_lyrics(song_name)
    
    # Perform sentiment analysis on the lyrics
    sentiment_scores = sid.polarity_scores(lyrics)
    
    # Return the sentiment scores
    return sentiment_scores

# Define Gradio interface
interface = gr.Interface(
    fn=analyze_song_sentiment,
    inputs="text",
    outputs="json",
    title="Song Sentiment Analyzer",
    description="Enter the name of a song and get the sentiment analysis of its lyrics."
)


#make a public url for the gradio app
interface.launch(share=True)



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mariaelena/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://0630f0f215fc2ae95a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
